In [1]:
import numpy as np
import random
from datetime import datetime
from datetime import timedelta
import pandas as pd
from statistics import mean
from loguru import logger
import uuid
import json
import pytz
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
from statsmodels.formula.api import ols
plt.style.use('seaborn')

In [2]:
# global constants
filename = '20200715-E7S2WK4HFP-br.jsonl'

def parse_file(n_lines=None):
    with open(filename) as infile:
        if n_lines is not None: # only read at most n_lines
            file_iterator = itertools.islice(infile,n_lines,n_lines+500000)
        else:
            file_iterator = infile
        all_dict = list(map(json.loads,file_iterator))
        logger.info("Loaded {} rows",len(all_dict))
        return all_dict
    
def create_frame(br_object):
    deals = br_object['Bidrequest']['imp'][0]['pmp']['deals']
    for deal in deals:
        if deal['id'] == "E7S2WK4HFP":
            price = deal['bidfloor'] 
    record={
        'ts': br_object['Timestamp_status_receive_ms']/1000,
        'TZ': br_object['Timezone'],
        'imps': br_object['Imps'],
        'CPM': price,
        'Date': datetime.fromtimestamp(br_object['Timestamp_status_receive_ms']/1000)
        }
    record['price'] = (record['imps'] * record['CPM']) / 1000
    return record

In [ ]:
%%time

# We have to load 6 000 000 lines
first = True
N_LINES = 0
while N_LINES < 5500000:
    if first:
        first = False
        all_dicts = parse_file(n_lines=N_LINES)
        records = list(map(create_frame, all_dicts))
        df = pd.DataFrame.from_records(records).sort_values("ts",ascending=True)
    else:
        N_LINES += 500000
        all_dicts = parse_file(n_lines=N_LINES)
        records = list(map(create_frame, all_dicts))
        df = df.append(pd.DataFrame.from_records(records).sort_values("ts",ascending=True))  

In [ ]:
def win_proba(probability):
    return random.random() < probability
def delay_notif():
    if random.random() < 0.90:
        delay = random.randint(2,60)
    else:
        delay = random.randint(60,900)
    return delay

In [ ]:
df['win'] = df.apply(lambda x: win_proba(0.95), axis = 1)
df['seconds_notif'] = df.apply(lambda x: delay_notif(), axis = 1)

In [ ]:
def gen_prop_lr(br_object):
    aggr = br_object.imps.groupby([br_object.index.date,br_object.index.weekday,br_object.index.hour]).sum()
    aggr.index.names = ['date','weekday','hour']
    aggr = aggr.reset_index()
    model = ols('imps ~ C(weekday) + C(hour)', data=aggr).fit()
    weekday_list = range(7)
    weekday_list = list(itertools.chain.from_iterable(itertools.repeat(x, 24) for x in weekday_list))
    hour_list = list()
    for i in range(7):
        for z in range(24):
            hour_list.append(z)
    df_fitting = pd.DataFrame({'weekday':weekday_list,'hour':hour_list})
    prediction = model.predict(df_fitting)
    df_fitting['fitted'] = prediction
    pattern = df_fitting.pivot_table('fitted', index = df_fitting.hour, columns=df_fitting.weekday)
    line, col = pattern.shape
    for i in range(col):
        pattern.iloc[:,i] = pattern.iloc[:,i]*100/pattern.iloc[:,i].sum()
    return pattern

In [ ]:
def gen_prop_lr_hour(br_object):
    aggr = br_object.imps.groupby([br_object.index.date,br_object.index.hour]).sum().reset_index()
    aggr.columns = ['date','hour','imps']
    model = ols('imps ~ C(hour)', data=aggr).fit()
    hour_list = list()
    for z in range(24):
        hour_list.append(z)
    df_fitting = pd.DataFrame({'hour':hour_list})
    prediction = model.predict(df_fitting)
    df_fitting['fitted'] = prediction
    df_fitting.index = df_fitting.hour
    del df_fitting['hour']
    df_fitting.iloc[:,0] = df_fitting.iloc[:,0]*100/df_fitting.iloc[:,0].sum()
    return df_fitting

In [ ]:
def meta_prop(data):
    if data.empty:
        unif = True
        without_weekday = False
        prop = 1/24
    elif set(data.index.hour.unique()) != set(range(24)):
        unif = True
        without_weekday = False
        prop = 1/24
    else:
        diff = data.tail(1).index - data.head(1).index
        if diff.days < 7:
            unif = False
            without_weekday = True
            prop = gen_prop_lr_hour(data)
        elif diff.days == 7:
            unif = False
            without_weekday = False
            prop = gen_prop_lr(data)
    return (prop, unif, without_weekday)

In [ ]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.building = list()
        # Impossible day and hour to initialize the setup
        self.day = 0
        self.engaged_budget = 0
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la BR
        weekday = datetime.fromtimestamp(ts).weekday()
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        hour = datetime.fromtimestamp(ts).hour
        # If we begin a new day, we reset variables
        if self.day != day:
            if not self.building:
                self.building_data = pd.DataFrame.from_records(self.building)
            else:
                self.building_data = pd.DataFrame.from_records(self.building, index='Date')
            self.current_hour = -1
            self.remaining_budget_hour = 0
            self.remaining_budget = self.daily_budget
            self.spent_budget = 0
            self.surplus_hour = 0
            self.BT = [0]
            self.acceleration = [{'ts':datetime(year,month,day,0,0,0),
                                  'A':0}]
            self.speed = [{'ts':datetime(year,month,day,0,0,0),
                                  'S':0}]
            self.size_acceleration = 1
            self.sum_acceleration = 0
            self.size_speed = 1
            self.sum_speed = 0
            self.prop_table, self.unif, self.without_weekday = meta_prop(self.building_data)
        self.day = day
        # Changement of hour
        while hour != self.current_hour:
            self.current_hour += 1 
            self.remaining_hours = 24 - self.current_hour
            # Evolutive target
            self.surplus_hour += self.remaining_budget_hour / self.remaining_hours
            if self.unif:
                self.budget_hour = self.prop_table * self.daily_budget + self.surplus_hour
            elif self.without_weekday:
                self.budget_hour = (self.prop_table.iloc[self.current_hour, 0]/100)*self.daily_budget + self.surplus_hour
            else:
                self.budget_hour = (self.prop_table.iloc[self.current_hour, weekday]/100)*self.daily_budget + self.surplus_hour
            self.target = self.budget_hour/3600
            self.spent_hour = 0
            self.remaining_budget_hour = self.budget_hour - self.spent_hour
        # Remaining time before the end of the hour
        end_hour = datetime(year,month,day,hour,59,59,999999)
        remaining_time =  datetime.timestamp(end_hour) - ts
        # Average acceleration and speed
        created_time = self.acceleration[-1]['ts'] - timedelta(minutes=30)
        while self.acceleration[0]['ts'] < created_time:
            self.size_acceleration -= 1 
            self.sum_acceleration += self.acceleration[0]['A']
            del self.acceleration[0]
        try:
            average_acceleration = self.sum_acceleration / self.size_acceleration
        except ZeroDivisionError:
            average_acceleration = 0
        while self.speed[0]['ts'] < created_time:
            self.size_speed -= 1 
            self.sum_speed += self.speed[0]['S']
            del self.speed[0]
        try:
            average_speed = self.sum_speed / self.size_speed
        except ZeroDivisionError:
            average_speed = 0
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        alpha = average_acceleration
        # Calculation of bt
        try:
            bt = self.remaining_budget_hour * ((1 + alpha * average_speed) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        self.BT.append(bt)
        # Calculation of vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed.append({'ts':datetime.fromtimestamp(ts),
                          'S': vt})
        at = self.speed[-1]['S'] - self.speed[-2]['S']
        self.acceleration.append({'ts':datetime.fromtimestamp(ts),
                          'A': at})
        # Buying decision
        if (bt >= self.target) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
            self.spent_hour += price
        else:
            buying = False
        self.remaining_budget_hour = self.budget_hour - self.spent_hour

        return buying     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']
                self.spent_hour -= ev['br_price']

In [ ]:
pacing = Algo(daily_budget=3000, nb_hours_day=24)

In [ ]:
%%time
records = list()
pending_notifications = list()
day = 8
for current_ts, row in data_et.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        # Send remaining notifications
        pacing.send_pending_notifications()
        records[-1]['engaged'] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        records[-1]['remaining'] = pacing.remaining_budget
        records[-1]['spent'] = pacing.spent_budget
        
    # Receive BR
    buying = pacing.buying_decision(row['ts'], row['price'])
    # Making a decision
    if buying:
        #Buying
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    record = {
        'buying':buying,
        'remaining':pacing.remaining_budget,
        'spent':pacing.spent_budget,
        'engaged':pacing.engaged_budget
    }
    records.append(record)
    pacing.building.append({'Date':current_ts,'imps':row['imps']})
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
records[-1]['engaged'] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
records[-1]['remaining'] = pacing.remaining_budget
records[-1]['spent'] = pacing.spent_budget
pacing_df = pd.DataFrame.from_records(records)
new_df_et = pd.concat([data_et.reset_index(),pacing_df], axis=1, ignore_index=True)
new_df_et.columns = ['Date','ts','TZ','imps','CPM','price','win','seconds_notif',
                 'buying','remaining','spent','engaged']
new_df_et.set_index('Date', inplace=True)